# Drop [postgres_air] database
## ⚠️ Important Warning
Kindly run/open this file in Anaconda Jupyter Notebook. Not in Azure Data Studio

In [2]:
!psql -h localhost -U postgres -c 'drop database postgres_air;'

DETAIL:  The database was created using collation version 2.35, but the operating system provides version 2.39.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE postgres REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
DROP DATABASE


# Create empty [postgres_air] database

In [3]:
!psql -h localhost -U postgres -c 'create database postgres_air;'

DETAIL:  The database was created using collation version 2.35, but the operating system provides version 2.39.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE postgres REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
CREATE DATABASE


# Restore data for [postgres_air] from dump

In [4]:
!pg_restore -h localhost -d postgres_air -U postgres /stale-storage/Softwares/PostgreSQL/PostgreSQL-Sample-Dbs/postgres_air/postgres_air_2023.backup

## Set default schema to postgres_air

In [17]:
!psql -h localhost -d postgres_air -U postgres -c "ALTER ROLE postgres SET search_path TO postgres_air, public;"
!psql -h localhost -d postgres_air -U postgres -c "SHOW search_path;"

ALTER ROLE
     search_path      
----------------------
 postgres_air, public
(1 row)



# Validate row counts in tables

In [6]:
!psql -d postgres_air -f 'qry-get-postgres_air-tables-n-rowcounts.sql'

 table_schema |     table_name      |  rows_n  
--------------+---------------------+----------
 postgres_air | boarding_pass_large | 75880473
 postgres_air | boarding_pass       | 25293491
 postgres_air | booking_leg         | 17893566
 postgres_air | passenger           | 16313693
 postgres_air | booking             |  5643216
 postgres_air | account             |   864962
 postgres_air | flight              |   683178
 postgres_air | phone               |   407444
 postgres_air | frequent_flyer      |   128346
 postgres_air | airport             |      692
 postgres_air | aircraft            |       12
(11 rows)



# Create Initial Indexes

In [7]:
!psql -d postgres_air -f 'setup-postgres_air-indexes.sql'

SET
psql:setup-postgres_air-indexes.sql:5: ERROR:  relation "flight_departure_airport" already exists
psql:setup-postgres_air-indexes.sql:6: ERROR:  relation "flight_scheduled_departure" already exists
psql:setup-postgres_air-indexes.sql:7: ERROR:  relation "flight_update_ts" already exists
psql:setup-postgres_air-indexes.sql:8: ERROR:  relation "booking_leg_booking_id" already exists
psql:setup-postgres_air-indexes.sql:9: ERROR:  relation "booking_leg_update_ts" already exists
psql:setup-postgres_air-indexes.sql:10: ERROR:  relation "account_last_name" already exists


## Get me indexes on tables now

In [ ]:
!psql -d postgres_air -f 'qry-get-postgres_air-index-n-types.sql'

   table_name   |         index_name         | is_unique | is_primary | index_type |      index_keys       | is_functional | is_partial 
----------------+----------------------------+-----------+------------+------------+-----------------------+---------------+------------
 account        | account_pkey               | t         | t          | btree      | {account_id}          | f             | f
 aircraft       | aircraft_pkey              | t         | t          | btree      | {code}                | f             | f
 airport        | airport_pkey               | t         | t          | btree      | {airport_code}        | f             | f
 boarding_pass  | boarding_pass_pkey         | t         | t          | btree      | {pass_id}             | f             | f
 booking        | booking_booking_ref_key    | t         | f          | btree      | {booking_ref}         | f             | f
 booking_leg    | booking_leg_pkey           | t         | t          | btree      | {booki

In [8]:
!psql -d postgres_air -f 'qry-get-postgres_air-index-n-size.sql'

SET
      tablename      |           indexname            |   num_rows    | table_size | index_size | unique 
---------------------+--------------------------------+---------------+------------+------------+--------
 account             | account_last_name              |        864962 | 65 MB      | 6080 kB    | N
 account             | account_pkey                   |        864962 | 65 MB      | 19 MB      | Y
 aircraft            | aircraft_pkey                  |            12 | 8192 bytes | 16 kB      | Y
 airport             | airport_pkey                   |           692 | 80 kB      | 32 kB      | Y
 boarding_pass       | boarding_pass__booking_leg_id  | 2.5293492e+07 | 2038 MB    | 365 MB     | N
 boarding_pass       | boarding_pass__passenger_id    | 2.5293492e+07 | 2038 MB    | 358 MB     | N
 boarding_pass       | boarding_pass_pkey             | 2.5293492e+07 | 2038 MB    | 542 MB     | Y
 boarding_pass       | boarding_pass_update_ts        | 2.5293492e+07 | 2038 MB    |